# Lemmatiser

In [40]:
# FrenchLefffLemmatizer à installer si absent de l'env : 
# !pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git

# Télécharger packages si absents : 
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('punkt')

In [41]:
from pathlib import Path
from collections import defaultdict
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem import WordNetLemmatizer

In [42]:
# Lemmatiseur français
lemmatizer_fr = FrenchLefffLemmatizer()
# Lemmatiseur anglais
lemmatizer_en = WordNetLemmatizer()

# Lecture des corpus

In [43]:
# Chemins corpus
path_emea = Path("../data/Emea")
path_europarl = Path("../data/Europarl/")

# Création dico pour save corpus avec leurs noms de fichiers
corpus_list = defaultdict(list)

# Parcourir chaque fichier du dossier EMEA ou des sous-dossiers
for file in path_emea.glob('**/*tok.true.clean*'):
    with open(file, 'r') as file:
        f1 = file.read()
        # Extraire seulement le nom de fichiers comme clés 
        corpus_list[file.name.split('/')[-1]] = f1.split('\n')

# Parcourir chaque fichier du dossier Europarl ou des sous-dossiers
for file in path_europarl.glob('**/*tok.true.clean*'):
    with open(file, 'r') as file:
        f2 = file.read()
        # Extraire seulement le nom de fichiers comme clés 
        corpus_list[file.name.split('/')[-1]] = f2.split('\n')

# Lemmatisation

In [44]:
# Fonction pour lemmatiser un texte
def lemma_text(text: str, lang: str) -> str:
    """
    Lemmatisation d'un texte dans la langue spécifiée.

    Cette fonction lemmatise en utilisant NLTK pour l'anglais et le français.
    Français : lemmatisation de base effectuée avec lemmatizer_fr de FrenchLefffLemmatizer. 
    Anglais : POS tags utilisés pour lemmatisation plus précise. 
    
    Args:
        text (str): Texte à lemmatiser.
        lang (str): Langue du texte. En format code de langue ('en' et 'fr')

    Returns:
        str: Texte lemmatisé.

    Raises:
        ValueError: Si la langue renseignée n'est ni 'en' ni 'fr'.
    """

    # Lemmatisation de chaque mot avec son POS tag
    lem_words = []

    if lang == "fr":
        
        # Tokeniser texte en mots
        words = nltk.word_tokenize(text)
            
        # Lemmatisation du mot avec son POS tag
        lem_words = [lemmatizer_fr.lemmatize(word) for word in words]
        

    elif lang == "en":
        
        # Tokeniser texte en mots
        words = nltk.word_tokenize(text)
    
        # Obtenir POS tags des mots
        pos_tags = nltk.pos_tag(words)
        
        for word, pos_tag in pos_tags:
            # Convertir POS tags Penn Treebank en format WordNet
            pos_tag = pos_tag[0].lower()
            pos_tag = pos_tag if pos_tag in ['a', 'n', 'v'] else 'n'  # Convertir les POS tags inconnus en 'n' (nom)
    
            
            # Lemmatisation du mot avec son POS tag
            lemma = lemmatizer_en.lemmatize(word, pos=pos_tag)
            lem_words.append(lemma)

        
    # Jointure des mots lemmatisés en une seule chaîne de caractères
    lem_text = ' '.join(lem_words)
    
    return lem_text

In [45]:
# Création dico pour stocker phrases lemmatisées
lem_corpus = defaultdict(list)

# Parcourir chaque corpus
for i, j in corpus_list.items():
    # Création liste pour stocker phrases lemmatisées
    lem_sents: list = []

    # Pour français
    if i.endswith('fr'):
        # Lemmatiser chaque phrase
        for sent in j:
            # Lemmatiser phrase en français
            lem_sent = lemma_text(sent, 'fr')
            # Ajouter phrase lemmatisée à la liste
            lem_sents.append(lem_sent)
        # Stocker liste de phrases lemmatisées dans dico
        lem_corpus[i] = lem_sents

    # Pour anglais
    elif i.endswith('en'):
        # Lemmatiser chaque phrase
        for sent in j:
            # Lemmatiser phrase en anglais
            lem_sent = lemma_text(sent, 'en')
            # Ajouter phrase lemmatisée à la liste
            lem_sents.append(lem_sent)
        # Stocker liste de phrases lemmatisées dans dico
        lem_corpus[i] = lem_sents

In [47]:
# Exporter les corpus lemmatisés
for i, j in lem_corpus.items():
    with open(f'../data/lemmatised/lemma_{i}', 'w', encoding='utf-8') as file:
        file.write('\n'.join(j))